<a href="https://colab.research.google.com/github/rebekah1124/BigCon_NSshop/blob/master/3_modeling_add.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np

# 한글 인코딩 해결
import matplotlib as mpl
import matplotlib.font_manager as fm
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings(action='ignore')

import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import minmax_scale, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import check_array

In [ ]:
# MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

**모델링 시 유의사항**  
- `broadcastTime` / `date` / `time` / `minute` / `product_code` 빼고 돌리기  
- mothercode : ?  
- product : 일단 냅두되 모델 돌릴 땐 사용하지 X  
- (2) showtime : 그냥돌려보기 / 상품명별로 합쳐보기 (데이터셋 두 가지로 저장)  
- (2) category : OneHotEncoding / 카테고리별로 모델 돌려보거나 다 합쳐서 돌려보거나  
- (2) max_rating : 일단 NaN 다 0으로 / 혹시 같은 상품인데 max_rating값 있으면 그걸로 채워보기  
- avg_rating : 일단 넣어보되 빼는 것도 생각  
- amount : 예측해야할것 (예측변수를 order로 할거면 얘는 빼고돌리기)  
- day / week / isWeekend / isHoliday / timeslot / hour : 그냥 두기  
- (3) price : 걍 / 로그변환 / 상품군별로 standard scale  
- (2) order : y값 예측을 얘로 해보자? / 피쳐로 넣을거면 분당 판매량  
- temp : 최소/최고기온 (월별 minMaxScale)  
- precipitation : numeric (월별 minMaxScale)  
  
*경우의 수가 꽤 많으니까 주석처리를 해제했다 안했다 하는식으로 최대한 간편하게 경우의 수를 나눠 돌려볼 수 있도록 코드 구성 -> 각 경우의 수에 대해 mape 잘 기록해두기..*  
  
*추후 merging 등에서 에러나지 않도록 중간중간마다 index reset하고 순서 맞는지 확인하기*

# Read Dataset  
- train test datasets

In [ ]:
train = pd.read_csv('retail_extended.csv') 
test = pd.read_csv('test_extended.csv')

# Sanity Check  
- train data와 test data의 형식 같은지 체크  
- NaN값 없는지  
- column 나열 순서 동일한지

In [ ]:
print(train.shape)
train.columns

(35382, 40)


Index(['broadcastTime', 'showtime', 'mothercode', 'product', 'category',
       'max_rating', 'amount', 'date', 'month', 'day', 'week', 'isWeekend',
       'isHoliday', 'season', 'dayofweek', 'timeslot', 'time', 'hour',
       'minute', 'product_code', 'price', 'order', 'avg_rating', 'payment',
       'gender', 'isPremium', 'sub_category', 'precipitation', 'avg_temp',
       'min_temp', 'max_temp', 'setOrNot', 'showtime_sum', 'order_per_min',
       'log_amount', 'log_price', 'std_amount', 'std_price', 'minmax_amount',
       'minmax_price'],
      dtype='object')

In [ ]:
print(test.shape)
test.columns
# Does not include...
# avg_temp / min_temp / max_temp : Not implemented yet...
# max_rating / order / avg_rating / order_per_min / log_amount / std_amount / minmax_amount

(2716, 30)


Index(['broadcastTime', 'showtime', 'mothercode', 'product', 'category',
       'amount', 'date', 'month', 'day', 'week', 'isWeekend', 'isHoliday',
       'season', 'dayofweek', 'timeslot', 'time', 'hour', 'minute',
       'product_code', 'price', 'payment', 'gender', 'isPremium',
       'sub_category', 'precipitation', 'setOrNot', 'showtime_sum',
       'log_price', 'std_price', 'minmax_price'],
      dtype='object')

In [ ]:
train.isna().sum()

broadcastTime        0
showtime             0
mothercode           0
product              0
category             0
max_rating       10910
amount               0
date                 0
month                0
day                  0
week                 0
isWeekend            0
isHoliday            0
season               0
dayofweek            0
timeslot             0
time                 0
hour                 0
minute               0
product_code         0
price                0
order                0
avg_rating           0
payment          24324
gender           31111
isPremium            0
sub_category      1677
precipitation        0
avg_temp             0
min_temp             0
max_temp             0
setOrNot             0
showtime_sum         0
order_per_min        0
log_amount           0
log_price            0
std_amount           0
std_price            0
minmax_amount        0
minmax_price         0
dtype: int64

In [ ]:
test.isna().sum()

broadcastTime       0
showtime            0
mothercode          0
product             0
category            0
amount           2716
date                0
month               0
day                 0
week                0
isWeekend           0
isHoliday           0
season              0
dayofweek           0
timeslot            0
time                0
hour                0
minute              0
product_code        0
price               0
payment          2332
gender           2506
isPremium           0
sub_category      472
precipitation       0
setOrNot            0
showtime_sum        0
log_price           0
std_price           0
minmax_price        0
dtype: int64

# Define Feature & Response Variable  
제외 변수 : `broadcastTime` / `product` / `mothercode` / `date` / `time` / `minute` / `product_code` / `brand` / `max_rating` / 일단 강수량만 포함  
  
경우의 수(16가지)  
1. 무엇을 할지 (카테고리별 vs. 전체) 2가지  
2. Y변수를 무엇으로 할지 (취급액 vs. 주문량) 4가지  
     취급액 & Price Scaling(그대로 vs. 로그변환 vs. 상품군별 standard scaling)  
3. Temp/precipitation Scaling (그대로 vs. MinMaxScale) 2가지  

**케이스 나눌 때 특히 두 번째 경우 showtime이랑 최종 예측값 어떻게 계산해야 하는지 매우 잘 생각하기! 저는 넘 헷갈려용...:/**

In [ ]:
# (price, amount), (log_price, log_amount), (minmax_price, minmax_amount), (std_price, std_amount) 중 상황에 맞게 선택
feature = ['showtime', 'category', 'month', 'day', 'week', 'isWeekend', 'isHoliday',
           'season', 'dayofweek', 'timeslot', 'hour', 'price', 'payment', 'gender',
           'isPremium', 'sub_category', 'precipitation', 'setOrNot', 'showtime_sum']
           # 'log_price', 'std_price', 'minmax_price']
# 추후에 미세먼지랑 전국 기온도 완료되면 추가하기
response = 'amount'
# response = 'order'

In [ ]:
# 가능한 원데이터 건들지 않는 방향으로
tmp_train = train[feature]
tmp_train = tmp_train.reset_index(drop=True)

tmp_test = test[feature]
tmp_test = tmp_test.reset_index(drop=True)

# Scaling  
- `temp` / `precipitation` : 월별로 MinMaxScale  
- `category` / `season` / `dayofweek` / `payment` / `gender` / `sub_category` : OneHotEncoding

In [ ]:
# ohe : train
tmp_train.dayofweek = pd.Categorical(tmp_train.dayofweek)
ohe_tmp = pd.get_dummies(tmp_train[['category' , 'season', 'dayofweek', 'payment', 'gender', 'sub_category']])
ohe_tmp

,category_가구,category_가전,category_건강기능,category_농수축,category_생활용품,category_속옷,category_의류,category_이미용,category_잡화,category_주방,...,sub_category_티셔츠,sub_category_패딩,sub_category_패치,sub_category_팬츠,sub_category_프라이팬,sub_category_해산물,sub_category_행거,sub_category_헤어,sub_category_혈압계,sub_category_히터
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35377,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
35378,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
35379,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
35380,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# train : Join OHE dataframe and drop original feature columns
tmp_train = tmp_train.join(ohe_tmp)
tmp_train = tmp_train.drop(columns=['category' , 'season', 'dayofweek', 'payment',
                                      'gender', 'sub_category'])

In [ ]:
# sanity check
sum(tmp_train.isna().sum() != 0)

0

In [ ]:
# ohe : test
tmp_test.dayofweek = pd.Categorical(tmp_test.dayofweek)
ohe_tmp = pd.get_dummies(tmp_test[['category' , 'season', 'dayofweek', 'payment', 'gender', 'sub_category']])
ohe_tmp

,category_가구,category_가전,category_건강기능,category_농수축,category_생활용품,category_속옷,category_의류,category_이미용,category_잡화,category_주방,...,sub_category_커튼,sub_category_클렌징,sub_category_탁자,sub_category_탕,sub_category_티셔츠,sub_category_패치,sub_category_팬츠,sub_category_프라이팬,sub_category_해산물,sub_category_헤어
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2712,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2713,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2714,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
# 문제ㅠㅠㅠ
# sub_category가 58개 차이나고 season도 평가데이터에는 한 개라서 스케일링 했을 때 칼럼 수 차이남..
# 근데 이건 모델링이랑 병행하면서 수정해도 될듯
# MAPE 구하려면 train 데이터 validation set으로 쪼개서 봐야하니까 당장엔 test data로 평가하지 않아도 되긴 함
print(train['sub_category'].nunique())
print(test['sub_category'].nunique())

118
60


In [ ]:
mms=MinMaxScaler()
tmp_train[["min_temp", "max_temp","precipitation"]] = mms.fit_transform(train[["min_temp", "max_temp","precipitation"]])
#test에는 implement 안했다길래..
#tmp_test[["min_temp", "max_temp","precipitation"]] = mms.fit_transform(test[["min_temp", "max_temp","precipitation"]])

# Train-Validation set split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(tmp_train, train[response], test_size=0.3)
print(x_train.shape)
print(x_val.shape)

(24767, 157)
(10615, 157)


In [ ]:
x_test = tmp_test
y_test = test[response]

In [ ]:
sum(tmp_train.isna().sum() != 0)

0

# Modeling  

In [ ]:
# RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(random_state=1)

In [ ]:
# HyperParameter 조정 해보기 (Bayesian Optimizer)
forest.fit(x_train, y_train)

RandomForestRegressor(random_state=1)

In [ ]:
pred = forest.predict(x_val)

In [ ]:
# Sanity check for 0 division
sum(pred == 0)
sum(y_test == 0)

0

In [ ]:
mape = mean_absolute_percentage_error(y_val, pred)
print('MAPE :', mape)
# 각 경우와 MAPE 기록하는거 잊지 말기!

MAPE : 89.30569795689081
